In [ ]:
%matplotlib inline
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import wget
import os
import zipfile
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
from numpy import random
from scipy.stats import binom
import scipy.stats as stats
from statsmodels.tsa.arima_process import ArmaProcess
from scipy.special import digamma,polygamma
from scipy.linalg import sqrtm
from tqdm import tqdm
from datetime import datetime

In [ ]:
from nonparam_compositional_cpd.nonparametric_beta_ar_change_point_tester import NonparametricBetaARChangePointTester
from nonparam_compositional_cpd.threshold_generator import ThresholdGenerator

# data downloading and preprocessing

# Visualization

In [ ]:
save_adj_dk = pd.read_csv('adj_dk_2010_2020.csv')

save_adj_dk['date'] = [str(save_adj_dk.year.iloc[i])+'-'+str(save_adj_dk.month.iloc[i]) for i in range(len(save_adj_dk))]
save_adj_dk.date = pd.to_datetime(save_adj_dk.date)

In [ ]:
save_adj_dk.plot("date", "per", label="alcohol involved crash rate")
plt.gcf().autofmt_xdate()
plt.grid()
plt.xlabel('Date')
plt.ylabel('Alcohol involved crash rate')

# Nonparametric monitoring

In [ ]:
yrstart_in = 2010
yrend_in   = 2018
yrstart_mon= 2019
yrend_mon  = 2020

In [ ]:
m = (yrend_in- yrstart_in+1)*12
Nm = (yrend_mon- yrstart_mon+1)*12
N = Nm/m

per_adj = save_adj_dk[save_adj_dk.year>=yrstart_in].per_adj

grid = 5
length = 20
nIteration = 10000
gamma_vector = [0,0.25,0.4]
A = np.eye(grid)/grid
y_list = per_adj[:m]
est = ThresholdGenerator(grid=grid, length=length)
thresholds, quantiles, x_quants, quantile_map, A = est.threshold_generation_given_seq(y_list, N, nIteration, A=A, gamma_values=gamma_vector)


# thres, quan_list, x_quan_list, quan_map, A = threshold_generation_given_seq(y_list, N,grid, length, nIteration, A)


In [ ]:
est = NonparametricBetaARChangePointTester(m, N, grid, thresholds, quantile_map, gamma_vector, A)
results = est.run_test(per_adj)
results

In [ ]:
save_adj_dk.iloc[results[0][0]]

# CPI-AB processing and change point detection

In [ ]:
# CPI alcohol beverage and accidents involved drinking
filename = 'CUSR.csv'
cpi = pd.read_csv(filename)

In [ ]:
cpi.DATE = pd.to_datetime(cpi.DATE)
cpi['year'] = cpi.DATE.dt.year
cpi['month']= cpi.DATE.dt.month
cpi.columns = ['date', 'cpi_per','year','month']

cpi.date = pd.to_datetime(cpi.date)

In [ ]:
cpi_sub = cpi.merge(save_adj_dk, on=["year", "month"], how="inner")

In [ ]:
cpi_sub.plot(x="date_x", y="cpi_per", label="CPI-AB")

# Format x-axis dates
plt.gcf().autofmt_xdate()

# Set major ticks to show every year
ax = plt.gca()

# Add grid and labels
plt.grid()
plt.xlabel('Date')
plt.ylabel('Percentage change from year ago (%)')